##Identify the Digits
Automatic digit recognition is of popular interest today. Deep Learning techniques makes it possible for object recognition in image data . This practice problem is meant to give you a kick start in deep learning. As usual, we will not only provide you with the challenge and a solution checker, but also a set of tutorials to get you off the ground!

The data set used for this problem is from the populat MNIST data set. Developed by Yann LeCun, Corina Cortes and Christopher Burger for evaluating machine learning model on the handwritten digit classification problem.

In [ ]:
!wget --header="Host: datahack-prod.s3.amazonaws.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://datahack.analyticsvidhya.com/" "https://datahack-prod.s3.amazonaws.com/train_file/Train_UQcUa52.zip" -c -O 'Train_UQcUa52.zip'

--2020-10-29 18:08:34--  https://datahack-prod.s3.amazonaws.com/train_file/Train_UQcUa52.zip
Resolving datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)... 52.219.66.84
Connecting to datahack-prod.s3.amazonaws.com (datahack-prod.s3.amazonaws.com)|52.219.66.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52075589 (50M) [application/zip]
Saving to: ‘Train_UQcUa52.zip’

Train_UQcUa52.zip   100%[===================>]  49.66M  10.4MB/s    in 5.4s    

2020-10-29 18:08:40 (9.12 MB/s) - ‘Train_UQcUa52.zip’ saved [52075589/52075589]



In [ ]:
!ls

sample_data  Train_UQcUa52.zip


In [ ]:
!unzip Train_UQcUa52.zip

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import keras
from keras import Model
from keras.layers import Conv2D,Dense,MaxPooling2D,AveragePooling2D,BatchNormalization,Input,Dropout,Flatten
from keras.optimizers import Adam
from keras.models import Sequential
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

# for creating validation set
from sklearn.model_selection import train_test_split

# for evaluating the model
from sklearn.metrics import accuracy_score

In [ ]:
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
# https://gist.github.com/greydanus/f6eee59eaf1d90fcb3b534a25362cea4
# https://stackoverflow.com/a/14434334 # this function is used to update the plots for each epoch and error
def plt_dynamic(x, vy, ty, ax, colors=['b']):
  ax.plot(x, vy, 'b', label="Validation Loss")
  ax.plot(x, ty, 'r', label="Train Loss")
  plt.legend()
  plt.grid()
  fig.canvas.draw()

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('Test_fCbTej3_0j1gHmj.csv')

In [ ]:
train.tail()

,filename,label
48995,48995.png,2
48996,48996.png,4
48997,48997.png,9
48998,48998.png,3
48999,48999.png,0


In [ ]:
y_train=train.label

In [ ]:
y_train

0        4
1        9
2        1
3        7
4        3
        ..
48995    2
48996    4
48997    9
48998    3
48999    0
Name: label, Length: 49000, dtype: int64

In [ ]:
y_train=keras.utils.to_categorical(y_train,num_classes)

In [ ]:
num_classes=train.label.nunique()
print(num_classes)

10


In [ ]:
test.tail()

,filename
20995,69995.png
20996,69996.png
20997,69997.png
20998,69998.png
20999,69999.png


In [ ]:
train_path=os.path.join(os.getcwd(),'Images/train/')
print(train_path)
test_path=os.path.join(os.getcwd(),'Images/test/')
print(test_path)

/content/Images/train/
/content/Images/test/


In [ ]:
train_img=[]
for img in tqdm(train['filename']):
  image=imread(train_path+img)
  image=image/255.
  image=resize(image,(28,28,1),mode='constant')
  image=image.astype('float')
  train_img.append(image)
  

100%|██████████| 49000/49000 [00:40<00:00, 1221.15it/s]


In [ ]:
train_img=np.array(train_img)
train_img.shape

(49000, 28, 28, 1)

In [ ]:
test_img=[]
for img in tqdm(test['filename']):
  image=imread(test_path+img)
  image=image/255.
  image=resize(image,(28,28,1),mode='constant')
  image=image.astype('float')
  test_img.append(image)
test_img=np.array(test_img)
test_img.shape  

100%|██████████| 21000/21000 [00:16<00:00, 1250.30it/s]


(21000, 28, 28, 1)

In [ ]:
np.save('train_img.npy',train_img)
np.save('test_img.npy',test_img)

In [ ]:
X_train=np.load('./train_img.npy',allow_pickle=True)
X_test=np.load('./test_img.npy',allow_pickle=True)

In [ ]:
# Network Architecture
# input -> conv -> conv -> pooling -> conv -> conv -> pooling ->dropout-> FC -> output
# 16 16 32 32 512

inp_shape=X_test.shape[1:]
model=Sequential()
model.add(Conv2D(16,kernel_size=(3,3),padding='same',activation='relu',input_shape=inp_shape)) #using same padding
model.add(Conv2D(16,5,padding='same',activation='relu'))
model.add(MaxPooling2D(strides=2))
model.add(Conv2D(32,5,activation='relu',padding='same'))
model.add(Conv2D(32,5,activation='relu',padding='same'))
model.add(MaxPooling2D(strides=2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()  

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_31 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 28, 28, 16)        6416      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 14, 14, 32)        12832     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 14, 14, 32)        25632     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 7, 7, 32)         

In [ ]:
history=model.fit(X_train,y_train,epochs=15,batch_size=128,verbose=1,validation_split=0.2)

Epoch 1/15
307/307 [==============================] - 3s 9ms/step - loss: 0.2614 - accuracy: 0.9163 - val_loss: 0.0682 - val_accuracy: 0.9787
Epoch 2/15
307/307 [==============================] - 2s 8ms/step - loss: 0.0672 - accuracy: 0.9782 - val_loss: 0.0592 - val_accuracy: 0.9820
Epoch 3/15
307/307 [==============================] - 2s 7ms/step - loss: 0.0420 - accuracy: 0.9869 - val_loss: 0.0446 - val_accuracy: 0.9857
Epoch 4/15
307/307 [==============================] - 2s 8ms/step - loss: 0.0333 - accuracy: 0.9897 - val_loss: 0.0345 - val_accuracy: 0.9892
Epoch 5/15
307/307 [==============================] - 2s 7ms/step - loss: 0.0290 - accuracy: 0.9906 - val_loss: 0.0329 - val_accuracy: 0.9910
Epoch 6/15
307/307 [==============================] - 2s 7ms/step - loss: 0.0222 - accuracy: 0.9927 - val_loss: 0.0296 - val_accuracy: 0.9914
Epoch 7/15
307/307 [==============================] - 2s 8ms/step - loss: 0.0195 - accuracy: 0.9935 - val_loss: 0.0415 - val_accuracy: 0.9892
Epoch 

In [ ]:
history2=model.fit(X_train,y_train,epochs=30,batch_size=256,verbose=1,validation_split=0.25)

Epoch 1/30
144/144 [==============================] - 2s 15ms/step - loss: 0.3496 - accuracy: 0.8877 - val_loss: 0.0902 - val_accuracy: 0.9713
Epoch 2/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0768 - accuracy: 0.9750 - val_loss: 0.0565 - val_accuracy: 0.9823
Epoch 3/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0545 - accuracy: 0.9826 - val_loss: 0.0631 - val_accuracy: 0.9807
Epoch 4/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0399 - accuracy: 0.9869 - val_loss: 0.0465 - val_accuracy: 0.9853
Epoch 5/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0339 - accuracy: 0.9882 - val_loss: 0.0420 - val_accuracy: 0.9870
Epoch 6/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0270 - accuracy: 0.9907 - val_loss: 0.0380 - val_accuracy: 0.9896
Epoch 7/30
144/144 [==============================] - 2s 13ms/step - loss: 0.0222 - accuracy: 0.9926 - val_loss: 0.0388 - val_accuracy: 0.9888

In [ ]:
X_test=np.load('./test_img.npy',allow_pickle=True)# Network Architecture
# input -> conv -> polling -> conv -> polling -> conv -> polling ->dropout-> FC -> output
# 8 32 128 64
model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=inp_shape))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))# for the location invariance
model1.add(Conv2D(64, (3,3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2),strides=2))
model1.add(Dropout(0.9))
model1.add(Flatten())
model1.add(Dense(64, activation='relu'))
model1.add(Dense(num_classes, activation='softmax'))
model1.compile(loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy'])

model1.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 3, 3, 128)         73856     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 1, 1, 128)         0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1, 1, 128)        

In [ ]:
history2=model1.fit(X_train,y_train,epochs=30,verbose=1,validation_split=0.20,batch_size=128)

Epoch 1/30
307/307 [==============================] - 2s 6ms/step - loss: 1.6325 - accuracy: 0.3919 - val_loss: 0.7097 - val_accuracy: 0.8494
Epoch 2/30
307/307 [==============================] - 2s 5ms/step - loss: 1.1111 - accuracy: 0.5839 - val_loss: 0.4118 - val_accuracy: 0.9095
Epoch 3/30
307/307 [==============================] - 2s 5ms/step - loss: 0.9568 - accuracy: 0.6448 - val_loss: 0.2883 - val_accuracy: 0.9359
Epoch 4/30
307/307 [==============================] - 2s 5ms/step - loss: 0.8626 - accuracy: 0.6857 - val_loss: 0.2790 - val_accuracy: 0.9360
Epoch 5/30
307/307 [==============================] - 2s 5ms/step - loss: 0.8060 - accuracy: 0.7071 - val_loss: 0.2382 - val_accuracy: 0.9404
Epoch 6/30
307/307 [==============================] - 2s 5ms/step - loss: 0.7541 - accuracy: 0.7276 - val_loss: 0.2050 - val_accuracy: 0.9494
Epoch 7/30
307/307 [==============================] - 2s 5ms/step - loss: 0.7045 - accuracy: 0.7466 - val_loss: 0.1852 - val_accuracy: 0.9534
Epoch 

This model has not performed well.
we will use first model 

In [ ]:
pred =np.array(model.predict(X_test))


In [ ]:
pred

array([[6.4498579e-25, 4.0007149e-18, 8.9591040e-20, ..., 1.5699406e-16,
        1.9459938e-20, 1.4998182e-19],
       [1.0000000e+00, 7.2433094e-14, 3.1988835e-11, ..., 4.7379042e-16,
        1.0028973e-12, 1.0242691e-11],
       [1.7701690e-05, 2.5618079e-11, 5.4091409e-08, ..., 7.0005754e-08,
        2.2611146e-06, 9.9997640e-01],
       ...,
       [2.2670352e-13, 2.8482740e-18, 1.3518777e-15, ..., 9.9225124e-25,
        9.8218500e-10, 1.6196993e-19],
       [3.9621896e-11, 1.9836280e-14, 7.8765992e-15, ..., 5.2805887e-19,
        9.8923714e-10, 3.5995552e-17],
       [8.6779484e-16, 3.7193874e-18, 1.0000000e+00, ..., 4.3823165e-22,
        2.4916688e-22, 2.5258925e-21]], dtype=float32)

In [ ]:
predictions=[]
for i in pred:
  predictions.append(np.argmax(i))

In [ ]:
predictions

[4,
 0,
 9,
 7,
 9,
 6,
 6,
 7,
 0,
 4,
 2,
 8,
 4,
 6,
 1,
 2,
 9,
 6,
 1,
 4,
 0,
 8,
 4,
 3,
 7,
 7,
 5,
 1,
 6,
 4,
 1,
 1,
 2,
 7,
 1,
 8,
 0,
 3,
 2,
 4,
 3,
 1,
 8,
 7,
 7,
 7,
 3,
 5,
 0,
 0,
 2,
 5,
 6,
 5,
 1,
 7,
 2,
 6,
 7,
 5,
 9,
 3,
 9,
 4,
 3,
 1,
 6,
 1,
 2,
 2,
 5,
 9,
 3,
 0,
 2,
 2,
 1,
 3,
 5,
 4,
 7,
 5,
 4,
 7,
 8,
 6,
 6,
 8,
 6,
 3,
 5,
 5,
 5,
 3,
 6,
 9,
 7,
 6,
 9,
 3,
 9,
 2,
 4,
 6,
 2,
 1,
 3,
 5,
 1,
 3,
 2,
 1,
 1,
 0,
 5,
 9,
 4,
 5,
 2,
 1,
 5,
 8,
 9,
 9,
 9,
 0,
 6,
 4,
 8,
 6,
 3,
 4,
 5,
 4,
 6,
 6,
 6,
 7,
 2,
 0,
 1,
 2,
 8,
 8,
 2,
 7,
 0,
 6,
 1,
 5,
 8,
 9,
 6,
 9,
 8,
 1,
 1,
 1,
 8,
 1,
 0,
 0,
 5,
 8,
 6,
 3,
 4,
 0,
 4,
 1,
 7,
 5,
 7,
 4,
 2,
 4,
 1,
 8,
 1,
 7,
 1,
 2,
 0,
 8,
 0,
 8,
 5,
 5,
 0,
 9,
 9,
 5,
 9,
 5,
 6,
 0,
 4,
 2,
 1,
 4,
 3,
 7,
 2,
 5,
 1,
 0,
 7,
 7,
 1,
 5,
 8,
 3,
 2,
 3,
 8,
 5,
 3,
 9,
 4,
 5,
 3,
 1,
 1,
 1,
 0,
 8,
 0,
 4,
 0,
 2,
 5,
 9,
 9,
 1,
 7,
 4,
 4,
 1,
 0,
 4,
 3,
 1,
 3,
 8,
 9,
 5,
 6,
 8,
 0,
 6,


In [ ]:
sub=test['filename']

In [ ]:
sub.head()

0    49000.png
1    49001.png
2    49002.png
3    49003.png
4    49004.png
Name: filename, dtype: object

In [ ]:
sub['label']=predictions

In [ ]:
predict = pd.DataFrame(data=predictions ,columns=["label"])


In [ ]:
sub = test['filename']
DT = pd.merge(sub , predict, on=None, left_index= True,
    right_index=True)

In [ ]:
DT.to_csv('brahm_submssion_mnist.csv',index=False)